In [18]:
import os
import pandas as pd
import regex

In [2]:
# output files
outHitsWithMismatch = "basta-allinfo-mismatch.xlsx"

## Read auto-reactive sequences

In [3]:
myfile = "autoantibody_sequences.xlsx"

In [4]:
xl = pd.ExcelFile(myfile)
sheet_names = xl.sheet_names  # see all sheet names
print(sheet_names)

['B005', 'B007', 'B008', 'B009', 'B010', 'B011', '0843']

In [5]:
def read_excel_sheet(myfile, sheetname):
    df = pd.read_excel(myfile, sheet_name=sheetname, header=None)
    df.columns = ['sequence']
    df['sheet'] = sheetname
    return(df)

In [6]:
df_cdr3 = read_excel_sheet(myfile, sheet_names[0])
for sheetname in sheet_names[1:]:
    df_cdr3 = pd.concat([df_cdr3, read_excel_sheet(myfile, sheetname)])
df_cdr3 = df_cdr3.reset_index()
df_cdr3.tail()

,index,sequence,sheet
510,25,TCTVSGDSMSSHYWSWIRQPPGKGPEWIAYIYYSGSTNYNPSLRGR...,0843
511,26,TCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNSNPSLKSR...,0843
512,27,TCTVSGGSISSYYWSWIRQTPGKGLEWIGYIYDSGSTNYNPSLKSR...,0843
513,28,CARKAAGGPFDLWGRGTLVT,0843
514,29,>>0843_RF,0843


In [7]:
df_cdr3.head()

,index,sequence,sheet
0,0,>>B005_anti-Ro52,B005
1,1,YYCAKESSIAAALYNRFDHWGQGTLVTVSSASTK,B005
2,2,LSCAASGFTFSLYAMNWVRQAPGKGLEWVSGLFGSGGGTDYADSVR...,B005
3,3,LSCAASGFTFSSLNMNWVRQAPGKGLEWVSSISSSSSYKYYVDSVK...,B005
4,4,LSCAASGFTFRSYWMHWVRQAPGKGLVWVSRINSDGINTYYADSVR...,B005


In [8]:
# Remove the ">>some-name" rows
patternDel = ">>"
filter = df_cdr3['sequence'].str.contains(patternDel)
df_cdr3 = df_cdr3[~filter]
df_cdr3.head()

,index,sequence,sheet
1,1,YYCAKESSIAAALYNRFDHWGQGTLVTVSSASTK,B005
2,2,LSCAASGFTFSLYAMNWVRQAPGKGLEWVSGLFGSGGGTDYADSVR...,B005
3,3,LSCAASGFTFSSLNMNWVRQAPGKGLEWVSSISSSSSYKYYVDSVK...,B005
4,4,LSCAASGFTFRSYWMHWVRQAPGKGLVWVSRINSDGINTYYADSVR...,B005
5,5,LSCAASEFSFSKYWIHWVRHVPGKGLVWVSRINSDGINTYYADSVR...,B005


In [9]:
def f1(x): # remove first two amino acids if it starts with YY
    if x.startswith("YY"):
        return(x[2:])
    else:
        return(x)

def f2(x): # remove everything after the "VT" if the pattern "VTVS" is in the sequence
    x = x.split("VTVS")
    if len(x) > 2:
        print("WARNING: multiple times VTVS in sequence", x)
    return(x[0] + "VT")
    
df_cdr3['sequence_without_YY'] = [x for x in map(f1, df_cdr3['sequence'])]
df_cdr3['cdr3pep'] = [x for x in map(f2, df_cdr3['sequence_without_YY'])]
df_cdr3.head()

,index,sequence,sheet,sequence_without_YY,cdr3pep
1,1,YYCAKESSIAAALYNRFDHWGQGTLVTVSSASTK,B005,CAKESSIAAALYNRFDHWGQGTLVTVSSASTK,CAKESSIAAALYNRFDHWGQGTLVT
2,2,LSCAASGFTFSLYAMNWVRQAPGKGLEWVSGLFGSGGGTDYADSVR...,B005,LSCAASGFTFSLYAMNWVRQAPGKGLEWVSGLFGSGGGTDYADSVR...,LSCAASGFTFSLYAMNWVRQAPGKGLEWVSGLFGSGGGTDYADSVR...
3,3,LSCAASGFTFSSLNMNWVRQAPGKGLEWVSSISSSSSYKYYVDSVK...,B005,LSCAASGFTFSSLNMNWVRQAPGKGLEWVSSISSSSSYKYYVDSVK...,LSCAASGFTFSSLNMNWVRQAPGKGLEWVSSISSSSSYKYYVDSVK...
4,4,LSCAASGFTFRSYWMHWVRQAPGKGLVWVSRINSDGINTYYADSVR...,B005,LSCAASGFTFRSYWMHWVRQAPGKGLVWVSRINSDGINTYYADSVR...,LSCAASGFTFRSYWMHWVRQAPGKGLVWVSRINSDGINTYYADSVR...
5,5,LSCAASEFSFSKYWIHWVRHVPGKGLVWVSRINSDGINTYYADSVR...,B005,LSCAASEFSFSKYWIHWVRHVPGKGLVWVSRINSDGINTYYADSVR...,LSCAASEFSFSKYWIHWVRHVPGKGLVWVSRINSDGINTYYADSVR...


## Read all info files and lookup sequences, allow for mismatches

In [10]:
allinfo_files = [x for x in os.listdir(".") if x.endswith(".all_info.csv")]
allinfo_files[:10]

['B005-SG-B_S1_L001.assembled-ACGTACGT-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B007-BMDC-B_S4_L001.assembled-AGTCAGTC-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B007-Lnpre-B_S2_L001.assembled-ACTGACTG-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'BASTA-1467-B_S49_L001.assembled-ACTGACTG-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B007-Lnpost-B_S3_L001.assembled-AGCTAGCT-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B007-PBBMDC-1-B_S5_L001.assembled-ATCGATCG-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B007-PBBMDC-2-B_S6_L001.assembled-ATGCATGC-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B009-clone2-B_S56_L001.assembled-ACGTACGT-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B009-clone6-B_S57_L001.assembled-ACTGACTG-IGH_HUMAN-all_info.csv.rr.all_info.csv',
 'B009-clone10-B_S58_L001.assembled-AGCTAGCT-IGH_HUMAN-all_info.csv.rr.all_info.csv']

In [16]:
cdr3_list = list(set(df_cdr3['sequence']))
print("number of sequences to lookup:", len(cdr3_list))

number of sequences to lookup: 408


In [84]:
motif = "(" + "|".join(cdr3_list) + "){e<=1}"
motif = motif.replace(" ", "") # remove whitespace
motif = motif.replace("||", "|")
print(motif)
p = regex.compile(motif, regex.BESTMATCH)

(LSCAASGFTVSSNYMSWVRQAPGKGLEWVGRIKSKTDGGTTDYAAPVKGRFTISRDDSKTTLYLQMNSLKTEDTAVYYCTTGGLTKREAYDSSGYNRYWFDPWGQGTLVTVSSASTKGKTIPWREGGDT|LSCAASGFSLSDHYIDWVRQAPGKGLEWVGRTRNKANSYKTEFAASVKGRFTISRDDSKNSLFLEMSSLKTEDTAVYYCATHTKTTSLGYYGLDVWGQGTAVTVSSASTKGKGINWREGGDTTYVR|LSCAASGFTFTTYSMTWVRQAPGKGLEWISYITSTGRTIYYADSVKGRFTISRDNAKNSLFLQMNSLRAEDTALYYCAKENLVVAGGIRNWFDPWGQGTLVTVSSASTKGKXLVWREGGDTTYVN|LSLTCTVSGDSISSGNYYWSWIRQPAGKGPEWIGRIYTSGSGSANYNPSLKSRVTISGDKSKNQFFLKLTSVTAADTAVYYCARHGGLASTPLRYWGQGTLVTVSSASTKGKHKMWREGGDTSXLE|ISCKGSGYSFTSYWIGWVRQVPGKGLEWMGIIYPGDSDTRYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYCAREYYDYVWGSYRYSRFDPWGQGTLVTVSSASTKGKHPYWREGGDTDXLE|ISCKGSGYTFTSYDINWVRQATGQGLEWMGWMNPNSGNTGYAQKFQGRVTMTRNTSISTAYMELSSLRSEDTAVYYCAREDPTTTVTHRYFDYWGQGTLVTVSSASTKGKLIRWREGGDTYVRM|YYCARDSLRAPPLYFDYW|YYCSRDGGLREAFDVWGQGTMVTVSSASTK|LSCAASGFTFSNYGLNWVRQAPGKGLEWVSYINSGGTSKEYADSVRGRFTISRDNANNSLDLQMSSLRVEDTAVYYCARDWSGTYRAVNWGQGTLVTVSSASTKGKQEHWREGGDTTYVK|ISCKGSGYTFTGYYMHWVRQAPGQGLEWMGWINPNSGGTNYAQKFQGRVTMIRDTSIRTAYMELS

In [96]:
def lookupSubSequence(pep):
    # motif and p are global variables
    m = p.search(str(pep))
    if m is not None:   # a match is found
        match = m.group()
        
        # Lookup the original sequence
        motif_reverse = "(" + match + "){e<=1}"
        p_reverse = regex.compile(motif_reverse, regex.BESTMATCH)
        m_reverse = p_reverse.search(str(motif))
        orig_pattern = m_reverse.group()
        return(orig_pattern + "," + match)
    else:
        return(None)

In [97]:
def lookupSubSequencePerFile(df_allinfo):
    df_allinfo['hit'] = [x for x in map(lookupSubSequence, df_allinfo['pep'])]
    df_tmp = df_allinfo[df_allinfo['hit'].notna()]
    df_tmp["orig_seq"] = [x.split(",")[0] for x in df_tmp["hit"]]
    df_tmp["found_seq"] = [x.split(",")[1] for x in df_tmp["hit"]]
    return(df_tmp)

In [98]:
df_allinfo = pd.read_csv(allinfo_files[0], sep="\t")
sample_name, rest = allinfo_files[0].split("_L001")
df_allinfo["Sample"] = sample_name
df_lookup_long = lookupSubSequencePerFile(df_allinfo)
print(sample_name, len(df_lookup_long))

B005-SG-B_S1 2426


/data/home/bioinfo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/data/home/bioinfo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
for allinfo_file in allinfo_files[1:]:
    df_allinfo = pd.read_csv(allinfo_file, sep="\t")
    sample_name, rest = allinfo_file.split("_L001")
    df_allinfo["Sample"] = sample_name
    df_tmp = lookupSubSequencePerFile(df_allinfo)
    df_lookup_long = pd.concat([df_lookup_long, df_tmp])
    print(sample_name, len(df_tmp))
print("ALL", len(df_lookup_long))

In [ ]:
df_lookup_long.to_excel(outHitsWithMismatch)
print("Wrote", outHitsWithMismatch, "to disk")